In [1]:
import pickle
import numpy as np

# Charger les objets
with open("../models/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

with open("../models/label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

X_train = np.load("../models/X_train.npy")
X_test = np.load("../models/X_test.npy")
y_train = np.load("../models/y_train.npy")
y_test = np.load("../models/y_test.npy")
